In [3]:
import glob
import h5py
import pandas as pd

from src import io, roi, ttl, zdF
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

exp_date = '251014'
exp_num = '1701'

experiment_path = io.get_experiment(exp_date, exp_num)
print(experiment_path)
supervox_path = io.get_file(experiment_path, 'processed', '*supervoxels.h5')
labels_path = io.get_file(experiment_path, 'processed', '*labels.h5')
print(supervox_path, labels_path)

# open h5 and pull data
with h5py.File(labels_path, 'r') as labels_hf:
    cluster_labels = labels_hf['labels'][...]

with h5py.File(supervox_path,'r') as supervox_hf:
    ca_signal = supervox_hf['ca_signal'][...]
    fictrac_sp = supervox_hf['smoothed_speed'][...]
    fictrac_t = supervox_hf['fictrac_time'][...]
    camera_fr = supervox_hf['camera_fr'][...]
    scope_fr = supervox_hf['scope_fr'][...]
    brain_dim = supervox_hf['brain_dimensions'][...]

# maui time conversions
frames = [x for x in range(1, ca_signal.shape[-1])]
volume_per_s = scope_fr/ca_signal.shape[0] # volume rate in Hz
maui_time = [0] + [x / volume_per_s for x in frames]

brain_dim= brain_dim.reshape(-1)
# fictrac time conversions
fic_time = fictrac_t / camera_fr

In [4]:
exp_date = '251014'
exp_num = '1801'

experiment_path = io.get_experiment(exp_date, exp_num)
print(experiment_path)
green_path = io.get_file(experiment_path,'raw', 'channel_2.nii')
red_path = io.get_file(experiment_path,'raw', 'channel_1.nii')

/mnt/z/princess/data/251014_1801


In [ ]:
n_vols =300 # number of volumes to average across
# averaged_time = n_vols // volume_per_s
# print(fr'averaged over {averaged_time} s')

# load in green data
green_img = nib.load(green_path)
green_data = green_img.dataobj[...,:n_vols]
print(green_data.shape)

# calculate the mean signal over time within a slice
mean_green_data = np.mean(green_data, axis=3)
nii_save = io.save_nii(fr'{exp_num}_mean_GCaMP_{n_vols}volumes.nii', mean_green_data)

del green_data

In [ ]:
# load in red data
red_img = nib.load(red_path)
red_data = red_img.dataobj[...,:n_vols]
print(red_data.shape)

# calculate the mean signal over time within a slice
mean_red_data = np.mean(red_data, axis=3)
nii_save = io.save_nii(fr'{exp_num}_mean_Tomato_{n_vols}volumes.nii', mean_red_data)

del red_data

(256, 256, 31, 978)


In [ ]:
# for slice in range(mean_green_data.shape[2]):
for slice in range(10):
    fig, axes = plt.subplots(1, 2)
    # plt.title(fr'slice number {slice + 1}', loc='center')
    axes[0].imshow(mean_green_data[:,:,slice].T)
    axes[0].set_xticks([])
    axes[0].set_yticks([])
    axes[0].set_title(fr'slice n {slice+1}:     GCaMP6f', loc='left')
    axes[1].imshow(mean_red_data[:,:,slice].T)
    axes[1].set_title('Tomato signal')
    axes[1].set_xticks([])
    axes[1].set_yticks([])
    plt.tight_layout
    plt.show()

## older notes + testing

n_vols = 300 # to average over time
green_data = green_img.dataobj[...,:n_vols]
print(green_data.shape)

red_data = red_img.dataobj[...,:n_vols]
print(red_data.shape)

averaged_time = n_vols // volume_per_s
print(fr'averaged over {averaged_time} s')
# mean_image_data = np.mean(data, axis=3)

mean_green_data = np.mean(green_data, axis=3)
mean_red_data = np.mean(red_data, axis=3)
mean_green_data.shape